In [320]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("pandas")
install("numpy")
install("matplotlib")

In [321]:
import pandas as pd
import numpy as np
import json
import csv
import os
import math

In [322]:
df = pd.read_csv("data/04 extraction consolidation results_backup_sepEQI.csv", sep="|")

In [323]:
metadata = pd.read_csv("./data/search_results.csv", sep=";", encoding="latin-1")

In [324]:
data = df.merge(metadata, how='left', on='Title')

In [325]:
includedTitles = df['Title']

In [326]:
includedTitles = includedTitles.dropna().unique()

In [327]:
includedTitles = pd.DataFrame(includedTitles, columns=['Title'])

In [328]:
data = includedTitles.merge(metadata, how='left', on='Title').dropna(subset=['Title'])

In [329]:
data['Document Identifier'].unique()

array(['J', 'Article', 'IEEE Journals', 'IEEE Conferences',
       'Conference Paper', 'Conference paper', 'C', nan], dtype=object)

In [330]:
possibilities = ['J', 'Article', 'IEEE Journals', 'IEEE Conferences','Conference Paper', 'Conference paper', 'C']

In [331]:
tags = []
shorts = []

In [332]:
for index, row in data.iterrows():
    di = row['Document Identifier']

    if di == possibilities[0]:
        tags.append("@article")
        

    elif di == possibilities[1]:
        tags.append("@article")
        
    
    elif di == possibilities[2]:
        tags.append("@article")
    
    elif di == possibilities[3]:
        tags.append("@inproceedings")
    
    elif di == possibilities[4]:
        tags.append("@inproceedings")
    
    elif di == possibilities[5]:
        tags.append("@inproceedings")

    elif di == possibilities[6]:
        tags.append("@inproceedings")
    
    else:
        tags.append("@misc")
    


In [333]:
len(tags)

102

In [334]:
data2 = data.assign(tag= tags)

In [335]:
yearless = {'A digital twin-driven hybrid approach for the prediction of performance degradation in transmission unit of CNC machine tool':2022,
'A framework for Model-Driven Engineering of resilient software-controlled systems':2021,
'A framework for Model-Driven Engineering of resilient software-controlled systems':2021,
'A Methodology for Digital Twin Modeling and Deployment for Industry 4.0':2020,
'AML4DT: A Model-Driven Framework for Developing and Maintaining Digital Twins with AutomationML':2021,
'Closed-Loop Systems Engineering (CLOSE): Integrating Experimentable Digital Twins with the Model-Driven Engineering Process':2018,
'Combining Low-Code Programming and SDL-Based Modeling with Snap! in the Industry 4.0 Context':2021,
'DataTime: a Framework to smoothly Integrate Past, Present and Future into Models':2021,
'Digital Behavioral Twins for Safe Connected Cars':2018,
'Digital Twin of an Electrical Motor Based on Empirical Performance Model':2020,
'Ensuring consistency in scalable-detail models for DT-based control':2021,
'Generating customized low-code development platforms for digital twins':2022,
'Generative and Model-driven SDK development for the Industrie 4.0 Digital Twin':2021,
'Interface Development for Digital Twin of an Electric Motor Based on Empirical Performance Model':2022,
'Knowledge driven rapid development of white box digital twins for industrial plant systems':2021,
'Meta-model-based shop-floor digital twin architecture, modeling and application':2023,
'Model-Driven Development of a Digital Twin for Injection Molding':2020,
'Model-driven engineering city spaces via bidirectional model transformations':2021,
'Supporting Digital Twins for the Retrofit in Aviation by a Model-Driven Data Handling':2023,
'Supporting the Implementation of Digital Twins for IoT-Enhanced BPs':2023,
'The Framework for Designing Autonomous Cyber-Physical Multi-agent Systems for Adaptive Resource Management':2019,
'The OMiLAB Digital Innovation environment: Agile conceptual models to bridge business value with Digital and Physical Twins for Product-Service Systems development':2022,
'Towards a Model-Driven Architecture for Interactive Digital Twin Cockpits':2020,
'Using UML and OCL Models to Realize High-Level Digital Twins':2021}

In [336]:
for index, row in data2.iterrows():
    docid = row['Document Identifier']
    title = row['Title']
    year = row['Publication Year']
    name = row['Authors']
    if pd.isna(docid):
        di = ''
    else:
        di = docid[0:1]
    if pd.isna(name):
        out = ""
        #print(year, name, di)

        if year == 2018:
            out=di+str(18)+title
        elif year == 2019:
            out=di+str(19)+title
        elif year == 2020:
            out=di+str(20)+title
        elif year == 2021:
            out=di+str(21)+title
        elif year == 2022:
            out=di+str(22)+title
        elif year == 2023:
            out=di+str(23)+title
        elif year == 2024:
            out=di+str(24)+title
    
        else:
            if row['Title'] in yearless.keys():
                out = str(di)+ str(yearless[row['Title']])+ title
                data2.at[index,'Publication Year'] = yearless[row['Title']]
                #row['Publication Year'] = yearless[row['Title']]
            else:
                out=str(di)+title
            #print(row['Title'])
            #out=str(di)+title
        shorts.append(out)
        continue
    shname = name.replace(' ', '').replace('.','')[0:3]
    title = title.replace(' ', '')[0:3]
    
    out = ""
    #print(year, name, di)

    if year == 2018:
        out=di+shname+str(18)+title
    elif year == 2019:
        out=di+shname+str(19)+title
    elif year == 2020:
        out=shname+str(20)+title
    elif year == 2021:
        out=di+shname+str(21)+title
    elif year == 2022:
        out=di+shname+str(22)+title
    elif year == 2023:
        out=di+shname+str(23)+title
    elif year == 2024:
        out=di+shname+str(24)+title
    
    else:
        if row['Title'] in yearless.keys():
            out = shname+ str(yearless[row['Title']])+ title
            data2.at[index,'Publication Year'] = yearless[row['Title']]
        else:
            out=shname+title
    shorts.append(out)

In [337]:
data2 = data2.assign(shorts= shorts)

In [338]:
bibtext = ""
for index,row in data2.iterrows():
    bibtext+=row['tag']+ "{"+row['shorts']+"\n"

    if pd.notna(row['Authors']):
        bibtext += "\tauthor = {"+ str(row['Authors'])+"}"
    if pd.notna(row['Publication Year']):
        bibtext += ",\n\tyear = {"+ str(row['Publication Year'])+"}"
    if pd.notna(row['Title']):
        bibtext += ",\n\ttitle = {"+ str(row['Title'])+"}"
    if pd.notna(row['Publisher']):
        bibtext += ",\n\tpublisher = {"+ str(row['Publisher'])+"}"
    bibtext+="\n}\n"


In [339]:
import unicodedata
out11 = unicodedata.normalize('NFKD', bibtext).encode('ascii', 'ignore')

In [340]:
out11

b'@article{Yan2022Adi\n\tauthor = {Yang, X; Ran, Y; Zhang, GB; Wang, HW; Mu, ZY; Zhi, SG},\n\tyear = {2022.0},\n\ttitle = {A digital twin-driven hybrid approach for the prediction of performance degradation in transmission unit of CNC machine tool}\n}\n@article{APar21Afr\n\tauthor = {Parri J.; Patara F.; Sampietro S.; Vicario E.},\n\tyear = {2021.0},\n\ttitle = {A framework for Model-Driven Engineering of resilient software-controlled systems},\n\tpublisher = {Springer}\n}\n@article{Par2021Afr\n\tauthor = {Parri, J; Patara, F; Sampietro, S; Vicario, E},\n\tyear = {2021.0},\n\ttitle = {A framework for Model-Driven Engineering of resilient software-controlled systems}\n}\n@article{IGNS21AMe\n\tauthor = {G. N. Schroeder; C. Steinmetz; R. N. Rodrigues; R. V. B. Henriques; A. Rettberg; C. E. Pereira},\n\tyear = {2021.0},\n\ttitle = {A Methodology for Digital Twin Modeling and Deployment for Industry 4.0},\n\tpublisher = {IEEE}\n}\n@article{ASch21AMe\n\tauthor = {Schroeder G.N.; Steinmetz C.

In [341]:
with open("lib.bib", "w") as f:#, encoding="utf-8"
    f.write(out11.decode("ascii"))